In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import kagglehub


path = kagglehub.dataset_download("jay7080dev/rice-plant-diseases-dataset")
print("Path to dataset files:", path)


base_dir = os.path.join(path, "rice leaf diseases dataset")  


datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()} 
print("Class indices:", class_indices)
print("Class labels:", class_labels)


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)


loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

model.save("rice_disease_model.h5")
np.save("class_labels.npy", class_labels)  
print("Model and labels saved successfully.")

2024-10-29 21:59:51.233896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Path to dataset files: /Users/user/.cache/kagglehub/datasets/jay7080dev/rice-plant-diseases-dataset/versions/1
Found 3748 images belonging to 3 classes.
Found 936 images belonging to 3 classes.
Class indices: {'Bacterialblight': 0, 'Brownspot': 1, 'Leafsmut': 2}
Class labels: {0: 'Bacterialblight', 1: 'Brownspot', 2: 'Leafsmut'}
Epoch 1/30
117/117 [==============================] - 140s 1s/step - loss: 0.8899 - accuracy: 0.5724 - val_loss: 0.7925 - val_accuracy: 0.6433
Epoch 2/30
117/117 [==============================] - 107s 916ms/step - loss: 0.6406 - accuracy: 0.7465 - val_loss: 0.5269 - val_accuracy: 0.7834
Epoch 3/30
117/117 [==============================] - 109s 931ms/step - loss: 0.4777 - accuracy: 0.8213 - val_loss: 0.4275 - val_accuracy: 0.8394
Epoch 4/30
117/117 [==============================] - 108s 925ms/step - loss: 0.3956 - accuracy: 0.8490 - val_loss: 0.3688 - val_accuracy: 0.8524
Epoch 5/30
117/117 [==============================] - 117s 997ms/step - loss: 0.3745 - a

/Users/user/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model and labels saved successfully.
